In [0]:
from __future__ import print_function
import random as rand
import csv
import operator
import gc
import os
from datetime import datetime
from keras.callbacks import EarlyStopping
from keras.models import load_model
import keras.backend as K
from sklearn.metrics import log_loss
import numpy as np

if K.backend() == 'tensorflow':
    import tensorflow as tf
from setuptools import setup, find_packages

from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras import backend as K
from devol import DEvol, GenomeHandler
import pandas as pd
from pyspark.sql.types import *
import inspect
from typing import Callable, List
import pandas as pd
from pyspark.sql import DataFrame, Row, column
from pyspark.sql.functions import lit, pandas_udf, PandasUDFType, array
from pyspark.sql.types import FloatType

In [0]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available: 0

# DEvol On MNIST Results
### Genome Record

In [0]:
# File location and type
file_location = "/dbfs/FileStore/tables/results"
file_type = "parquet"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = pd.read_csv(file_location, sep = delimiter)

display(df)
genomes= df
genomes.index = genomes['Val Accuracy']
genomes = genomes.sort_index().drop(columns=['Val Accuracy', 'Val Loss'])
champion = genomes.tail( n = 1)
best_genome = champion.iloc[0].tolist()

Conv0 active,Conv0 num filters,Conv0 batch normalization,Conv0 activation,Conv0 dropout,Conv0 max pooling,Conv1 active,Conv1 num filters,Conv1 batch normalization,Conv1 activation,Conv1 dropout,Conv1 max pooling,Conv2 active,Conv2 num filters,Conv2 batch normalization,Conv2 activation,Conv2 dropout,Conv2 max pooling,Conv3 active,Conv3 num filters,Conv3 batch normalization,Conv3 activation,Conv3 dropout,Conv3 max pooling,Conv4 active,Conv4 num filters,Conv4 batch normalization,Conv4 activation,Conv4 dropout,Conv4 max pooling,Conv5 active,Conv5 num filters,Conv5 batch normalization,Conv5 activation,Conv5 dropout,Conv5 max pooling,Dense0 active,Dense0 num nodes,Dense0 batch normalization,Dense0 activation,Dense0 dropout,Optimizer,Val Loss,Val Accuracy
1,16,1,1,7,0,0,64,0,0,4,0,0,128,1,1,8,0,1,32,0,0,3,0,0,64,0,0,2,1,0,32,0,0,4,1,0,128,0,0,4,2,0.317605584859848,0.9057999849319458
1,128,1,1,0,0,1,128,0,0,1,0,0,8,0,1,8,0,1,128,1,1,10,2,0,64,1,0,4,1,1,8,1,0,2,2,0,16,1,0,5,0,0.06870068609714508,0.9785000085830688
1,8,1,0,8,0,0,16,0,1,6,2,0,32,1,1,4,0,1,128,1,0,7,1,1,8,1,0,10,2,1,16,0,0,3,1,1,16,0,0,7,2,1.395317554473877,0.8306999802589417
1,128,0,1,5,1,1,8,1,1,7,2,1,16,1,1,2,2,1,256,1,0,1,0,0,128,1,1,6,1,0,32,0,1,6,2,0,128,0,1,3,2,0.415926992893219,0.8693000078201294
1,64,0,0,10,2,0,64,1,0,8,0,1,256,0,0,6,0,0,64,1,0,3,1,0,32,0,1,7,1,1,128,0,1,0,2,0,128,0,1,5,1,0.052957788109779365,0.9829000234603882
1,8,1,1,7,0,0,32,0,1,5,0,1,128,1,1,9,1,1,256,1,0,8,1,0,64,1,0,9,0,0,256,1,0,4,0,0,128,1,1,5,3,1.953291654586792,0.33629998564720154
1,8,0,0,2,0,1,128,1,0,7,0,1,128,0,0,4,0,1,128,0,0,8,2,0,16,1,0,4,2,1,256,0,0,10,1,0,512,1,0,4,2,0.1299610733985901,0.9763000011444092
1,256,1,0,1,1,1,32,1,1,10,2,1,64,0,1,1,1,0,256,1,0,8,2,0,32,1,1,0,2,0,128,0,0,5,2,1,64,1,0,9,3,2.2727112770080566,0.2152000069618225
1,128,1,1,4,0,1,8,1,0,2,0,0,8,1,0,2,0,0,32,0,1,6,1,0,256,1,1,1,2,1,128,1,1,2,0,0,128,0,1,5,1,1.6694078445434568,0.9330000281333924
1,8,0,0,4,1,0,64,1,0,10,2,0,128,0,0,8,1,0,128,0,0,7,0,1,128,0,1,9,0,1,64,1,1,10,0,0,512,0,1,4,0,0.1610460877418518,0.9666000008583068


In [0]:
# File location and type
file_location = "/dbfs/FileStore/tables/results"
file_type = "parquet"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = "\n"

# The applied options are for CSV files. For other file types, these will be ignored.
df = pd.read_csv(file_location, sep = delimiter)
df.columns = ['genes']
display(df)
genomes= df

genes
"1,16,1,1,7,0,0,64,0,0,4,0,0,128,1,1,8,0,1,32,0,0,3,0,0,64,0,0,2,1,0,32,0,0,4,1,0,128,0,0,4,2,0.317605584859848,0.9057999849319458"
"1,128,1,1,0,0,1,128,0,0,1,0,0,8,0,1,8,0,1,128,1,1,10,2,0,64,1,0,4,1,1,8,1,0,2,2,0,16,1,0,5,0,0.06870068609714508,0.9785000085830688"
"1,8,1,0,8,0,0,16,0,1,6,2,0,32,1,1,4,0,1,128,1,0,7,1,1,8,1,0,10,2,1,16,0,0,3,1,1,16,0,0,7,2,1.395317554473877,0.8306999802589417"
"1,128,0,1,5,1,1,8,1,1,7,2,1,16,1,1,2,2,1,256,1,0,1,0,0,128,1,1,6,1,0,32,0,1,6,2,0,128,0,1,3,2,0.415926992893219,0.8693000078201294"
"1,64,0,0,10,2,0,64,1,0,8,0,1,256,0,0,6,0,0,64,1,0,3,1,0,32,0,1,7,1,1,128,0,1,0,2,0,128,0,1,5,1,0.05295778810977936,0.9829000234603882"
"1,8,1,1,7,0,0,32,0,1,5,0,1,128,1,1,9,1,1,256,1,0,8,1,0,64,1,0,9,0,0,256,1,0,4,0,0,128,1,1,5,3,1.953291654586792,0.33629998564720154"
"1,8,0,0,2,0,1,128,1,0,7,0,1,128,0,0,4,0,1,128,0,0,8,2,0,16,1,0,4,2,1,256,0,0,10,1,0,512,1,0,4,2,0.1299610733985901,0.9763000011444092"
"1,256,1,0,1,1,1,32,1,1,10,2,1,64,0,1,1,1,0,256,1,0,8,2,0,32,1,1,0,2,0,128,0,0,5,2,1,64,1,0,9,3,2.2727112770080566,0.2152000069618225"
"1,128,1,1,4,0,1,8,1,0,2,0,0,8,1,0,2,0,0,32,0,1,6,1,0,256,1,1,1,2,1,128,1,1,2,0,0,128,0,1,5,1,1.669407844543457,0.9330000281333923"
"1,8,0,0,4,1,0,64,1,0,10,2,0,128,0,0,8,1,0,128,0,0,7,0,1,128,0,1,9,0,1,64,1,1,10,0,0,512,0,1,4,0,0.1610460877418518,0.9666000008583069"


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
K.set_image_data_format("channels_last")

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
dataset = ((x_train, y_train), (x_test, y_test))

In [0]:
genome_handler = GenomeHandler(max_conv_layers=6, 
                               max_dense_layers=2, # includes final dense layer
                               max_filters=256,
                               max_dense_nodes=1024,
                               input_shape=x_train.shape[1:],
                               n_classes=10)

In [0]:
devol = DEvol(genome_handler)

Genome encoding and metric data stored at Fri Nov 27 05:16:00 2020.csv

In [0]:
import pickle 
pickle_file = open('/dbfs/FileStore/models/simple/pickle','rb')
retrieved_devol = pickle.load(pickle_file)


In [0]:
df.genes = df.genes.astype(str)
#df = df.reset_index()
df = df.tail(n=3)
df

,genes
124,"1,128,1,0,9,1,1,128,0,0,1,0,0,8,0,1,0,2,1,256,..."
125,"1,128,1,1,7,0,1,16,1,0,2,0,0,128,1,1,8,0,1,32,..."
126,"1,128,1,1,4,0,1,16,0,0,4,0,0,128,1,1,8,0,1,32,..."


In [0]:
for index, row in df.iterrows():
  print(row.genes)
  
  

1,128,1,0,9,1,1,128,0,0,1,0,0,8,0,1,0,2,1,256,1,0,5,2,1,8,1,1,7,1,1,8,1,1,0,2,0,64,1,1,2,2,0.8317346572875977,0.958899974822998
1,128,1,1,7,0,1,16,1,0,2,0,0,128,1,1,8,0,1,32,1,0,3,0,0,64,0,0,2,1,0,32,0,0,10,2,0,64,1,1,2,2,0.09838128834962845,0.9696999788284302
1,128,1,1,4,0,1,16,0,0,4,0,0,128,1,1,8,0,1,32,1,0,1,0,0,128,1,1,6,1,0,32,0,1,0,0,1,1024,0,0,10,2,0.07148057222366333,0.9767000079154968

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.6226 - accuracy: 0.0625 2/1688 [..............................] - ETA: 3:28 - loss: 2.5170 - accuracy: 0.0781 3/1688 [..............................] - ETA: 4:32 - loss: 2.5245 - accuracy: 0.0625 4/1688 [..............................] - ETA: 5:00 - loss: 2.4746 - accuracy: 0.0859 5/1688 [..............................] - ETA: 5:21 - loss: 2.5114 - accuracy: 0.0750 6/1688 [..............................] - ETA: 5:36 - loss: 2.5117 - accuracy: 0.0729 7/1688 [..............................] - ETA: 5:47 - loss: 2.5169 - accuracy: 0.0759 8/1688 [..............................] - ETA: 5:55 - loss: 2.4983 - accuracy: 0.0859 9/1688 [..............................] - ETA: 6:02 - loss: 2.5001 - accuracy: 0.0799 10/1688 [..............................] - ETA: 6:06 - loss: 2.4978 - accuracy: 0.0844 11/1688 [..............................] - ETA: 6:09 - loss: 2.4903 - accuracy: 0.0881 12/1688 [..............................] - ETA: 6:11 - loss: 2.4796 - accuracy: 0.0938 13/1688 [..............................] - ETA: 6:15 - loss: 2.4703 - accuracy: 0.0938 14/1688 [..............................] - ETA: 6:17 - loss: 2.4631 - accuracy: 0.1004 15/1688 [..............................] - ETA: 6:18 - loss: 2.4499 - accuracy: 0.1000 16/1688 [..............................] - ETA: 6:19 - loss: 2.4412 - accuracy: 0.1016 17/1688 [..............................] - ETA: 6:20 - loss: 2.4366 - accuracy: 0.1029 18/1688 [..............................] - ETA: 6:21 - loss: 2.4290 - accuracy: 0.1007 19/1688 [..............................] - ETA: 6:22 - loss: 2.4192 - accuracy: 0.1036 20/1688 [..............................] - ETA: 6:23 - loss: 2.4143 - accuracy: 0.1078 21/1688 [..............................] - ETA: 6:24 - loss: 2.4182 - accuracy: 0.1042 22/1688 [..............................] - ETA: 6:26 - loss: 2.4113 - accuracy: 0.1023 23/1688 [..............................] - ETA: 6:26 - loss: 2.4034 - accuracy: 0.1046

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.5381 - accuracy: 0.0938 2/1688 [..............................] - ETA: 3:30 - loss: 2.4951 - accuracy: 0.0625 3/1688 [..............................] - ETA: 4:32 - loss: 2.5327 - accuracy: 0.0625 4/1688 [..............................] - ETA: 5:14 - loss: 2.5441 - accuracy: 0.0547 5/1688 [..............................] - ETA: 5:48 - loss: 2.5881 - accuracy: 0.0562 6/1688 [..............................] - ETA: 5:58 - loss: 2.5736 - accuracy: 0.0573 7/1688 [..............................] - ETA: 6:05 - loss: 2.5654 - accuracy: 0.0536 8/1688 [..............................] - ETA: 6:11 - loss: 2.5515 - accuracy: 0.0664 9/1688 [..............................] - ETA: 6:16 - loss: 2.5381 - accuracy: 0.0660 10/1688 [..............................] - ETA: 6:18 - loss: 2.5282 - accuracy: 0.0688 11/1688 [..............................] - ETA: 6:24 - loss: 2.5062 - accuracy: 0.0909 12/1688 [..............................] - ETA: 6:26 - loss: 2.4978 - accuracy: 0.0911 13/1688 [..............................] - ETA: 6:28 - loss: 2.4853 - accuracy: 0.0865 14/1688 [..............................] - ETA: 6:29 - loss: 2.4711 - accuracy: 0.0893 15/1688 [..............................] - ETA: 6:30 - loss: 2.4628 - accuracy: 0.0854 16/1688 [..............................] - ETA: 6:29 - loss: 2.4532 - accuracy: 0.0898 17/1688 [..............................] - ETA: 6:30 - loss: 2.4466 - accuracy: 0.0919 18/1688 [..............................] - ETA: 6:30 - loss: 2.4405 - accuracy: 0.0868 19/1688 [..............................] - ETA: 6:32 - loss: 2.4418 - accuracy: 0.0855 20/1688 [..............................] - ETA: 6:32 - loss: 2.4385 - accuracy: 0.0875 21/1688 [..............................] - ETA: 6:32 - loss: 2.4339 - accuracy: 0.0893 22/1688 [..............................] - ETA: 6:34 - loss: 2.4309 - accuracy: 0.0923 23/1688 [..............................] - ETA: 6:34 - loss: 2.4247 - accuracy: 0.0924

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.2634 - accuracy: 0.0625 2/1688 [..............................] - ETA: 3:25 - loss: 2.4114 - accuracy: 0.1250 3/1688 [..............................] - ETA: 4:33 - loss: 2.5601 - accuracy: 0.1042 4/1688 [..............................] - ETA: 5:06 - loss: 2.5789 - accuracy: 0.0781 5/1688 [..............................] - ETA: 5:22 - loss: 2.5549 - accuracy: 0.0875 6/1688 [..............................] - ETA: 5:38 - loss: 2.6182 - accuracy: 0.0990 7/1688 [..............................] - ETA: 5:48 - loss: 2.6620 - accuracy: 0.0938 8/1688 [..............................] - ETA: 5:56 - loss: 2.6399 - accuracy: 0.0859 9/1688 [..............................] - ETA: 6:00 - loss: 2.6211 - accuracy: 0.0833 10/1688 [..............................] - ETA: 6:04 - loss: 2.5974 - accuracy: 0.0812 11/1688 [..............................] - ETA: 6:09 - loss: 2.5628 - accuracy: 0.0852 12/1688 [..............................] - ETA: 6:12 - loss: 2.5693 - accuracy: 0.0833 13/1688 [..............................] - ETA: 6:14 - loss: 2.5349 - accuracy: 0.0889 14/1688 [..............................] - ETA: 6:16 - loss: 2.5327 - accuracy: 0.0915 15/1688 [..............................] - ETA: 6:17 - loss: 2.5187 - accuracy: 0.0938 16/1688 [..............................] - ETA: 6:20 - loss: 2.5107 - accuracy: 0.1016 17/1688 [..............................] - ETA: 6:21 - loss: 2.4955 - accuracy: 0.1066 18/1688 [..............................] - ETA: 6:22 - loss: 2.4876 - accuracy: 0.1111 19/1688 [..............................] - ETA: 6:23 - loss: 2.4748 - accuracy: 0.1135 20/1688 [..............................] - ETA: 6:24 - loss: 2.4643 - accuracy: 0.1219 21/1688 [..............................] - ETA: 6:24 - loss: 2.4651 - accuracy: 0.1235 22/1688 [..............................] - ETA: 6:25 - loss: 2.4610 - accuracy: 0.1222 23/1688 [..............................] - ETA: 6:27 - loss: 2.4595 - accuracy: 0.1277

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.5088 - accuracy: 0.1562 2/1688 [..............................] - ETA: 3:26 - loss: 2.5147 - accuracy: 0.1094 3/1688 [..............................] - ETA: 4:32 - loss: 2.4801 - accuracy: 0.1250 4/1688 [..............................] - ETA: 5:06 - loss: 2.4930 - accuracy: 0.1250 5/1688 [..............................] - ETA: 5:26 - loss: 2.4618 - accuracy: 0.1312 6/1688 [..............................] - ETA: 5:40 - loss: 2.4771 - accuracy: 0.1146 7/1688 [..............................] - ETA: 5:51 - loss: 2.4724 - accuracy: 0.1161 8/1688 [..............................] - ETA: 5:58 - loss: 2.4639 - accuracy: 0.1133 9/1688 [..............................] - ETA: 6:05 - loss: 2.4562 - accuracy: 0.1111 10/1688 [..............................] - ETA: 6:09 - loss: 2.4437 - accuracy: 0.1125 11/1688 [..............................] - ETA: 6:13 - loss: 2.4422 - accuracy: 0.1080 12/1688 [..............................] - ETA: 6:16 - loss: 2.4422 - accuracy: 0.1042 13/1688 [..............................] - ETA: 6:22 - loss: 2.4396 - accuracy: 0.0986 14/1688 [..............................] - ETA: 6:22 - loss: 2.4376 - accuracy: 0.0938 15/1688 [..............................] - ETA: 6:25 - loss: 2.4305 - accuracy: 0.0917 16/1688 [..............................] - ETA: 6:26 - loss: 2.4285 - accuracy: 0.0918 17/1688 [..............................] - ETA: 6:27 - loss: 2.4293 - accuracy: 0.0901 18/1688 [..............................] - ETA: 6:27 - loss: 2.4276 - accuracy: 0.0868 19/1688 [..............................] - ETA: 6:27 - loss: 2.4209 - accuracy: 0.0855 20/1688 [..............................] - ETA: 6:28 - loss: 2.4145 - accuracy: 0.0891 21/1688 [..............................] - ETA: 6:29 - loss: 2.4074 - accuracy: 0.0923 22/1688 [..............................] - ETA: 6:30 - loss: 2.4074 - accuracy: 0.0909 23/1688 [..............................] - ETA: 6:30 - loss: 2.4011 - accuracy: 0.0951

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.4134 - accuracy: 0.0312 2/1688 [..............................] - ETA: 3:20 - loss: 2.3583 - accuracy: 0.1094 3/1688 [..............................] - ETA: 4:40 - loss: 2.4511 - accuracy: 0.1042 4/1688 [..............................] - ETA: 5:16 - loss: 2.4713 - accuracy: 0.1016 5/1688 [..............................] - ETA: 5:41 - loss: 2.4687 - accuracy: 0.1063 6/1688 [..............................] - ETA: 5:54 - loss: 2.4767 - accuracy: 0.1042 7/1688 [..............................] - ETA: 6:04 - loss: 2.4618 - accuracy: 0.1116 8/1688 [..............................] - ETA: 6:12 - loss: 2.4729 - accuracy: 0.1055 9/1688 [..............................] - ETA: 6:15 - loss: 2.4718 - accuracy: 0.1042 10/1688 [..............................] - ETA: 6:19 - loss: 2.4628 - accuracy: 0.1031 11/1688 [..............................] - ETA: 6:22 - loss: 2.4379 - accuracy: 0.1080 12/1688 [..............................] - ETA: 6:24 - loss: 2.4376 - accuracy: 0.1146 13/1688 [..............................] - ETA: 6:27 - loss: 2.4401 - accuracy: 0.1154 14/1688 [..............................] - ETA: 6:26 - loss: 2.4362 - accuracy: 0.1205 15/1688 [..............................] - ETA: 6:29 - loss: 2.4265 - accuracy: 0.1292 16/1688 [..............................] - ETA: 6:30 - loss: 2.4181 - accuracy: 0.1348 17/1688 [..............................] - ETA: 6:31 - loss: 2.4075 - accuracy: 0.1379 18/1688 [..............................] - ETA: 6:33 - loss: 2.3995 - accuracy: 0.1406 19/1688 [..............................] - ETA: 6:33 - loss: 2.3978 - accuracy: 0.1398 20/1688 [..............................] - ETA: 6:33 - loss: 2.3981 - accuracy: 0.1391 21/1688 [..............................] - ETA: 6:34 - loss: 2.3939 - accuracy: 0.1414 22/1688 [..............................] - ETA: 6:35 - loss: 2.3945 - accuracy: 0.1378 23/1688 [..............................] - ETA: 6:35 - loss: 2.3952 - accuracy: 0.1345

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.4015 - accuracy: 0.0938 2/1688 [..............................] - ETA: 3:21 - loss: 2.4110 - accuracy: 0.0625 3/1688 [..............................] - ETA: 4:29 - loss: 2.3997 - accuracy: 0.0729 4/1688 [..............................] - ETA: 5:05 - loss: 2.4031 - accuracy: 0.0859 5/1688 [..............................] - ETA: 5:25 - loss: 2.4120 - accuracy: 0.0875 6/1688 [..............................] - ETA: 5:39 - loss: 2.4029 - accuracy: 0.0781 7/1688 [..............................] - ETA: 5:53 - loss: 2.3877 - accuracy: 0.0804 8/1688 [..............................] - ETA: 6:00 - loss: 2.3768 - accuracy: 0.0781 9/1688 [..............................] - ETA: 6:03 - loss: 2.3817 - accuracy: 0.0833 10/1688 [..............................] - ETA: 6:05 - loss: 2.3878 - accuracy: 0.0781 11/1688 [..............................] - ETA: 6:09 - loss: 2.3850 - accuracy: 0.0795 12/1688 [..............................] - ETA: 6:13 - loss: 2.3767 - accuracy: 0.0859 13/1688 [..............................] - ETA: 6:15 - loss: 2.3709 - accuracy: 0.0938 14/1688 [..............................] - ETA: 6:17 - loss: 2.3735 - accuracy: 0.0915 15/1688 [..............................] - ETA: 6:20 - loss: 2.3685 - accuracy: 0.0958 16/1688 [..............................] - ETA: 6:21 - loss: 2.3685 - accuracy: 0.0938 17/1688 [..............................] - ETA: 6:21 - loss: 2.3617 - accuracy: 0.0938 18/1688 [..............................] - ETA: 6:23 - loss: 2.3564 - accuracy: 0.0990 19/1688 [..............................] - ETA: 6:25 - loss: 2.3450 - accuracy: 0.1069 20/1688 [..............................] - ETA: 6:25 - loss: 2.3419 - accuracy: 0.1094 21/1688 [..............................] - ETA: 6:26 - loss: 2.3404 - accuracy: 0.1131 22/1688 [..............................] - ETA: 6:26 - loss: 2.3381 - accuracy: 0.1094 23/1688 [..............................] - ETA: 6:28 - loss: 2.3339 - accuracy: 0.1114

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.7920 - accuracy: 0.0312 2/1688 [..............................] - ETA: 3:34 - loss: 2.6867 - accuracy: 0.0781 3/1688 [..............................] - ETA: 4:45 - loss: 2.6684 - accuracy: 0.1042 4/1688 [..............................] - ETA: 5:17 - loss: 2.6176 - accuracy: 0.1016 5/1688 [..............................] - ETA: 5:36 - loss: 2.5842 - accuracy: 0.1063 6/1688 [..............................] - ETA: 5:50 - loss: 2.5523 - accuracy: 0.1094 7/1688 [..............................] - ETA: 5:58 - loss: 2.5264 - accuracy: 0.1116 8/1688 [..............................] - ETA: 6:06 - loss: 2.5326 - accuracy: 0.1094 9/1688 [..............................] - ETA: 6:09 - loss: 2.5266 - accuracy: 0.1007 10/1688 [..............................] - ETA: 6:14 - loss: 2.5221 - accuracy: 0.1000 11/1688 [..............................] - ETA: 6:17 - loss: 2.5013 - accuracy: 0.1136 12/1688 [..............................] - ETA: 6:21 - loss: 2.4848 - accuracy: 0.1120 13/1688 [..............................] - ETA: 6:22 - loss: 2.4809 - accuracy: 0.1130 14/1688 [..............................] - ETA: 6:23 - loss: 2.4746 - accuracy: 0.1071 15/1688 [..............................] - ETA: 6:25 - loss: 2.4654 - accuracy: 0.1146 16/1688 [..............................] - ETA: 6:27 - loss: 2.4629 - accuracy: 0.1113 17/1688 [..............................] - ETA: 6:29 - loss: 2.4525 - accuracy: 0.1085 18/1688 [..............................] - ETA: 6:29 - loss: 2.4433 - accuracy: 0.1094 19/1688 [..............................] - ETA: 6:31 - loss: 2.4344 - accuracy: 0.1184 20/1688 [..............................] - ETA: 6:32 - loss: 2.4271 - accuracy: 0.1234 21/1688 [..............................] - ETA: 6:33 - loss: 2.4289 - accuracy: 0.1205 22/1688 [..............................] - ETA: 6:33 - loss: 2.4251 - accuracy: 0.1193 23/1688 [..............................] - ETA: 6:33 - loss: 2.4247 - accuracy: 0.1196

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 3.0493 - accuracy: 0.0938 2/1688 [..............................] - ETA: 3:36 - loss: 3.0781 - accuracy: 0.1094 3/1688 [..............................] - ETA: 4:48 - loss: 3.0781 - accuracy: 0.0833 4/1688 [..............................] - ETA: 5:23 - loss: 3.0046 - accuracy: 0.1016 5/1688 [..............................] - ETA: 5:46 - loss: 2.9628 - accuracy: 0.1000 6/1688 [..............................] - ETA: 5:54 - loss: 2.9252 - accuracy: 0.0990 7/1688 [..............................] - ETA: 6:01 - loss: 2.9180 - accuracy: 0.0938 8/1688 [..............................] - ETA: 6:08 - loss: 2.8500 - accuracy: 0.1055 9/1688 [..............................] - ETA: 6:15 - loss: 2.8214 - accuracy: 0.1076 10/1688 [..............................] - ETA: 6:17 - loss: 2.8110 - accuracy: 0.1125 11/1688 [..............................] - ETA: 6:19 - loss: 2.7787 - accuracy: 0.1193 12/1688 [..............................] - ETA: 6:22 - loss: 2.7645 - accuracy: 0.1172 13/1688 [..............................] - ETA: 6:24 - loss: 2.7595 - accuracy: 0.1178 14/1688 [..............................] - ETA: 6:26 - loss: 2.7411 - accuracy: 0.1183 15/1688 [..............................] - ETA: 6:28 - loss: 2.7411 - accuracy: 0.1104 16/1688 [..............................] - ETA: 6:29 - loss: 2.7234 - accuracy: 0.1074 17/1688 [..............................] - ETA: 6:32 - loss: 2.7098 - accuracy: 0.1066 18/1688 [..............................] - ETA: 6:33 - loss: 2.7056 - accuracy: 0.1059 19/1688 [..............................] - ETA: 6:35 - loss: 2.6847 - accuracy: 0.1086 20/1688 [..............................] - ETA: 6:35 - loss: 2.6676 - accuracy: 0.1094 21/1688 [..............................] - ETA: 6:36 - loss: 2.6516 - accuracy: 0.1116 22/1688 [..............................] - ETA: 6:37 - loss: 2.6401 - accuracy: 0.1122 23/1688 [..............................] - ETA: 6:37 - loss: 2.6367 - accuracy: 0.1101

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.4484 - accuracy: 0.1562 2/1688 [..............................] - ETA: 3:32 - loss: 2.4943 - accuracy: 0.1406 3/1688 [..............................] - ETA: 4:42 - loss: 2.4313 - accuracy: 0.1667 4/1688 [..............................] - ETA: 5:15 - loss: 2.4509 - accuracy: 0.1484 5/1688 [..............................] - ETA: 5:32 - loss: 2.4783 - accuracy: 0.1500 6/1688 [..............................] - ETA: 5:46 - loss: 2.4784 - accuracy: 0.1354 7/1688 [..............................] - ETA: 5:53 - loss: 2.4510 - accuracy: 0.1250 8/1688 [..............................] - ETA: 6:01 - loss: 2.4496 - accuracy: 0.1172 9/1688 [..............................] - ETA: 6:05 - loss: 2.4495 - accuracy: 0.1111 10/1688 [..............................] - ETA: 6:12 - loss: 2.4512 - accuracy: 0.1063 11/1688 [..............................] - ETA: 6:14 - loss: 2.4361 - accuracy: 0.1136 12/1688 [..............................] - ETA: 6:19 - loss: 2.4360 - accuracy: 0.1120 13/1688 [..............................] - ETA: 6:22 - loss: 2.4258 - accuracy: 0.1178 14/1688 [..............................] - ETA: 6:26 - loss: 2.4313 - accuracy: 0.1228 15/1688 [..............................] - ETA: 6:28 - loss: 2.4309 - accuracy: 0.1229 16/1688 [..............................] - ETA: 6:30 - loss: 2.4316 - accuracy: 0.1172 17/1688 [..............................] - ETA: 6:30 - loss: 2.4220 - accuracy: 0.1176 18/1688 [..............................] - ETA: 6:31 - loss: 2.4121 - accuracy: 0.1198 19/1688 [..............................] - ETA: 6:32 - loss: 2.3967 - accuracy: 0.1283 20/1688 [..............................] - ETA: 6:32 - loss: 2.3946 - accuracy: 0.1250 21/1688 [..............................] - ETA: 6:33 - loss: 2.3801 - accuracy: 0.1324 22/1688 [..............................] - ETA: 6:34 - loss: 2.3751 - accuracy: 0.1335 23/1688 [..............................] - ETA: 6:35 - loss: 2.3713 - accuracy: 0.1399

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.3068 - accuracy: 0.0938 2/1688 [..............................] - ETA: 3:38 - loss: 2.4321 - accuracy: 0.0781 3/1688 [..............................] - ETA: 4:43 - loss: 2.4836 - accuracy: 0.0938 4/1688 [..............................] - ETA: 5:14 - loss: 2.4664 - accuracy: 0.1016 5/1688 [..............................] - ETA: 5:35 - loss: 2.4043 - accuracy: 0.1250 6/1688 [..............................] - ETA: 5:48 - loss: 2.4156 - accuracy: 0.1198 7/1688 [..............................] - ETA: 5:58 - loss: 2.4115 - accuracy: 0.1205 8/1688 [..............................] - ETA: 6:04 - loss: 2.4238 - accuracy: 0.1133 9/1688 [..............................] - ETA: 6:09 - loss: 2.4248 - accuracy: 0.1111 10/1688 [..............................] - ETA: 6:13 - loss: 2.4211 - accuracy: 0.1187 11/1688 [..............................] - ETA: 6:16 - loss: 2.4313 - accuracy: 0.1136 12/1688 [..............................] - ETA: 6:19 - loss: 2.4217 - accuracy: 0.1172 13/1688 [..............................] - ETA: 6:20 - loss: 2.4225 - accuracy: 0.1106 14/1688 [..............................] - ETA: 6:24 - loss: 2.4154 - accuracy: 0.1094 15/1688 [..............................] - ETA: 6:25 - loss: 2.4160 - accuracy: 0.1042 16/1688 [..............................] - ETA: 6:27 - loss: 2.4144 - accuracy: 0.1035 17/1688 [..............................] - ETA: 6:28 - loss: 2.4101 - accuracy: 0.1029 18/1688 [..............................] - ETA: 6:29 - loss: 2.4055 - accuracy: 0.0990 19/1688 [..............................] - ETA: 6:30 - loss: 2.4032 - accuracy: 0.0987 20/1688 [..............................] - ETA: 6:29 - loss: 2.3984 - accuracy: 0.1000 21/1688 [..............................] - ETA: 6:30 - loss: 2.3963 - accuracy: 0.0997 22/1688 [..............................] - ETA: 6:32 - loss: 2.3913 - accuracy: 0.0980 23/1688 [..............................] - ETA: 6:32 - loss: 2.3850 - accuracy: 0.1005

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.5544 - accuracy: 0.0938 2/1688 [..............................] - ETA: 3:49 - loss: 2.4757 - accuracy: 0.0938 3/1688 [..............................] - ETA: 4:56 - loss: 2.5128 - accuracy: 0.0833 4/1688 [..............................] - ETA: 5:32 - loss: 2.5249 - accuracy: 0.0859 5/1688 [..............................] - ETA: 5:49 - loss: 2.5570 - accuracy: 0.0750 6/1688 [..............................] - ETA: 6:04 - loss: 2.5449 - accuracy: 0.0833 7/1688 [..............................] - ETA: 6:10 - loss: 2.5551 - accuracy: 0.0714 8/1688 [..............................] - ETA: 6:15 - loss: 2.5447 - accuracy: 0.0703 9/1688 [..............................] - ETA: 6:22 - loss: 2.5276 - accuracy: 0.0694 10/1688 [..............................] - ETA: 6:26 - loss: 2.5207 - accuracy: 0.0750 11/1688 [..............................] - ETA: 6:32 - loss: 2.5103 - accuracy: 0.0795 12/1688 [..............................] - ETA: 6:43 - loss: 2.5212 - accuracy: 0.0755 13/1688 [..............................] - ETA: 7:02 - loss: 2.5040 - accuracy: 0.0817 14/1688 [..............................] - ETA: 7:07 - loss: 2.5137 - accuracy: 0.0804 15/1688 [..............................] - ETA: 7:04 - loss: 2.4986 - accuracy: 0.0896 16/1688 [..............................] - ETA: 7:03 - loss: 2.4927 - accuracy: 0.0879 17/1688 [..............................] - ETA: 7:02 - loss: 2.4893 - accuracy: 0.0864 18/1688 [..............................] - ETA: 7:02 - loss: 2.4879 - accuracy: 0.0868 19/1688 [..............................] - ETA: 7:02 - loss: 2.4782 - accuracy: 0.0888 20/1688 [..............................] - ETA: 7:03 - loss: 2.4809 - accuracy: 0.0891 21/1688 [..............................] - ETA: 7:01 - loss: 2.4766 - accuracy: 0.0893 22/1688 [..............................] - ETA: 7:00 - loss: 2.4758 - accuracy: 0.0866 23/1688 [..............................] - ETA: 6:59 - loss: 2.4802 - accuracy: 0.0870

In [0]:

for index, row in df.iterrows():
    
    gene = row.genes
    gene = gene.split(sep = ',')[:-2]
    result = []
    for item in gene:
      result.append(int(item))

    if(genome_handler.is_compatible_genome(result)):
      model, loss, accuracy = retrieved_devol._evaluate(result,1)
      print("loss/acc: ", loss, accuracy, "\n")


1/1688 [..............................] - ETA: 0s - loss: 2.5633 - accuracy: 0.1250 2/1688 [..............................] - ETA: 3:18 - loss: 2.4847 - accuracy: 0.1406 3/1688 [..............................] - ETA: 4:27 - loss: 2.5568 - accuracy: 0.1146 4/1688 [..............................] - ETA: 5:07 - loss: 2.5557 - accuracy: 0.1094 5/1688 [..............................] - ETA: 5:26 - loss: 2.5367 - accuracy: 0.1000 6/1688 [..............................] - ETA: 5:41 - loss: 2.5066 - accuracy: 0.0990 7/1688 [..............................] - ETA: 5:55 - loss: 2.5138 - accuracy: 0.0982 8/1688 [..............................] - ETA: 6:01 - loss: 2.5211 - accuracy: 0.0898 9/1688 [..............................] - ETA: 6:07 - loss: 2.5044 - accuracy: 0.0972 10/1688 [..............................] - ETA: 6:10 - loss: 2.4871 - accuracy: 0.1063 11/1688 [..............................] - ETA: 6:12 - loss: 2.4722 - accuracy: 0.1108 12/1688 [..............................] - ETA: 6:15 - loss: 2.4785 - accuracy: 0.1068 13/1688 [..............................] - ETA: 6:18 - loss: 2.4709 - accuracy: 0.1058 14/1688 [..............................] - ETA: 6:20 - loss: 2.4637 - accuracy: 0.1027 15/1688 [..............................] - ETA: 6:21 - loss: 2.4525 - accuracy: 0.1063 16/1688 [..............................] - ETA: 6:21 - loss: 2.4421 - accuracy: 0.1055 17/1688 [..............................] - ETA: 6:23 - loss: 2.4385 - accuracy: 0.1048 18/1688 [..............................] - ETA: 6:25 - loss: 2.4421 - accuracy: 0.1007 19/1688 [..............................] - ETA: 6:26 - loss: 2.4425 - accuracy: 0.0970 20/1688 [..............................] - ETA: 6:26 - loss: 2.4380 - accuracy: 0.0953 21/1688 [..............................] - ETA: 6:26 - loss: 2.4364 - accuracy: 0.0967 22/1688 [..............................] - ETA: 6:27 - loss: 2.4251 - accuracy: 0.0994 23/1688 [..............................] - ETA: 6:27 - loss: 2.4200 - accuracy: 0.1019